In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
poem_prompt = ChatPromptTemplate.from_messages([
('system', 'You are a poet who enjoys writing poems with programming languages. You can create easy-to-understand poems that are both fun and educational.'),
('user', 'I want to make poem with {language}.'),
])
explainer_prompt = ChatPromptTemplate.from_messages([
('system', 'You are a teacher who enjoys explaining poem with programming languages to beginners. You can create easy-to-understand explanations that are both fun and educational.'),
('user', 'I want to explain {poem} in simple terms.'),
])

poet = poem_prompt | chat
explainer = explainer_prompt | chat
whole = {'poem' : poet} | explainer
whole.invoke({'language': 'Python'})

In the land of code, where Python reigns,
A language beloved for its simplicity and gains.
With indentation as its guide,
It helps programmers far and wide.

From variables to loops, it's a breeze,
To write code that flows with ease.
Functions and classes, oh so neat,
Python makes coding a joyful feat.

So let's raise a toast to this language so grand,
In the world of programming, it truly stands.
Python, dear Python, we sing your praise,
For making our coding days brighter in many ways.Sure! Let's break down the poem using programming concepts:

1. **Python**: In the land of code, Python is the main programming language that is loved for its simplicity and benefits. It's like the ruler of the coding world.

2. **Indentation**: Python uses indentation (spaces or tabs at the beginning of lines) to show the structure of the code. It helps programmers understand the flow of the program.

3. **Variables and Loops**: Python makes it easy to work with variables (containers for storing data) 

AIMessageChunk(content="Sure! Let's break down the poem using programming concepts:\n\n1. **Python**: In the land of code, Python is the main programming language that is loved for its simplicity and benefits. It's like the ruler of the coding world.\n\n2. **Indentation**: Python uses indentation (spaces or tabs at the beginning of lines) to show the structure of the code. It helps programmers understand the flow of the program.\n\n3. **Variables and Loops**: Python makes it easy to work with variables (containers for storing data) and loops (repeating a set of instructions) in a smooth and straightforward way.\n\n4. **Functions and Classes**: Python allows you to define functions (blocks of reusable code) and classes (blueprints for creating objects) in an organized and clean manner.\n\n5. **Coding Joy**: Writing code in Python is enjoyable and satisfying because of its simplicity and readability. It makes programming a fun and pleasant experience.\n\nIn summary, the poem celebrates P

In [3]:
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

examples = [
    {'movie': 'Lord of the Rings',
     'answer': '''
        Here is some information about Lord of the Rings:
        Director: Peter Jackson
        Genre: Fantasy
        Year: 2001
     '''},
    {'movie': 'The Matrix',
    'answer': '''
        Here is some information about The Matrix:
        Director: Lana Wachowski
        Genre: Science Fiction
        Year: 1999'
        '''
    },
    {'movie': 'The Shawshank Redemption',
    'answer': '''
        Here is some information about The Shawshank Redemption:
        Director: Frank Darabont
        Genre: Drama
        Year: 1994
        '''
    }
]
example_prompt = ChatPromptTemplate.from_messages([
    ('human', 'What do you know about {movie}?'),
    ('ai', '{answer}.'),
])



example_prompt = FewShotChatMessagePromptTemplate(
    examples = examples,
    example_prompt=example_prompt,
)


final_prompt = ChatPromptTemplate.from_messages([
    ('system', "You are a movie professional who enjoys tell about movies. But answer short and sweet."),
    example_prompt,
    ('user', "Tell me about {movie}."),
])



chain = final_prompt | chat
chain.invoke({"movie" : "Saving private Ryan"})

Here is some information about Saving Private Ryan:
Director: Steven Spielberg
Genre: War/Drama
Year: 1998

AIMessageChunk(content='Here is some information about Saving Private Ryan:\nDirector: Steven Spielberg\nGenre: War/Drama\nYear: 1998')

In [15]:
from langchain.prompts import FewShotChatMessagePromptTemplate, MessagesPlaceholder, ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationSummaryBufferMemory

chat = ChatOpenAI(
    temperature=.1,
)

example_prompt = FewShotChatMessagePromptTemplate(
    examples=[
        {'movie': 'Topgun', 'answer': '🛩️👨‍✈️🔥'},
        {'movie': 'GodFather', 'answer': '👨‍👨‍👦🔫🍝'},
    ],
    example_prompt=ChatPromptTemplate.from_messages([
        ('human', 'Abstract the {movie} to 3 emoji'),
        ('ai', '{answer}.'),
    ]),
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', "You are a movie professional who abstrcat whole movies to 3 emoji."),
        MessagesPlaceholder(variable_name='history'),
        example_prompt,
        ('user', "Abstrcact the {movie} to 3 emoji."),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})['history']



memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=120,
    return_messages=True,
)



chain = RunnablePassthrough.assign(history=load_memory)|final_prompt | chat

def invoke_chain(movie):
    res = chain.invoke({'movie': movie})
    print(res)
    memory.save_context({'input': movie},{'output': str(res)})

invoke_chain('The Matrix')


content='🕶️💊💻.'


In [16]:
invoke_chain('Lord of the Rings')

content='🧝🧙🗡️.'


In [17]:
invoke_chain('What was the previous two movies i asked?')

content='🕶️💊💻🧝🧙🗡️.'
